In [ ]:
from langgraph.graph import StateGraph, END

#1. 분류 노드
def classification(state):
    text = state.get("user_infput", "")
    if any (word in text for word in ["좋아요", "네", "좋다", "응"]):
        label = "positive"
    elif any (word in text for word in ["싫어요", "아니요", "싫다", "별로"]):
        label = "negative"
    else:
        label = "neutral"
    return {**state, "label":label}

#2. 응답 노드
def positive_response(state):
    return {**state, "response": "좋게 생각해주셔서 감사합니다!"}
def negative_response(state):
    return {**state, "response": "무엇이 불편하셨나요?"}
def neutral_response(state):
    return {**state, "response": "조금 더 자세히 말씀해 주세요."}

# 그래프 정의
graph = StateGraph(dict)
graph.add_node("classification", classification)
graph.add_node("positive_response", positive_response)
graph.add_node("negative_response", negative_response)
graph.add_node("neutral_response", neutral_response)

graph.set_entry_point("classification")

def get_label(state):
    return state.get("label", "")

graph.add_conditional_edges("classification",get_label, {
                            "positive":"positive_response",
                            "negative":"negative_response",
                            "neutral":"neutral_response"
})
graph.add_edge("positive_response", END)
graph.add_edge("negative_response", END)
graph.add_edge("neutral_response", END)

# 그래프 실행
app = graph.compile()
user_input = input("한글 입력: ")
finanl_state = app.invoke({"user_input": user_input})
print("응답:", finanl_state.get("response", ""))



응답: 무엇이 불편하셨나요?
